# Human Activity Tracking w/ Smartphone Dataset 
## UCI Maching Learning Repository - Dataset Original 
## Kaggle Simplified Version - Versión simplificada 

Abstract: Recordings of subjects performing activities while carrying inertial sensors.

Dataset Original: https://archive.ics.uci.edu/ml/datasets/human+activity+recognition+using+smartphones

Dataset simplificado (el que vamos a usar): https://www.kaggle.com/mboaglio/simplifiedhuarus

Clasificaremos la actividad que se estaba realizando: WALKING, WALKING_UPSTAIRS, WALKING_DOWNSTAIRS, SITTING, STANDING y LAYING

Para cada registro en el dataset se proporciona:
- Aceleración triaxial del acelerómetro (aceleración total) y la aceleración corporal estimada.
- Velocidad angular triaxial desde el giroscopio.
- Un vector de características 561 con variables de dominio de tiempo y frecuencia.
- Su etiqueta de actividad.
- Un identificador del sujeto que realizó el experimento.

# Download / split / upload a S3

In [1]:
%%sh
wget -q -N https://github.com/mboaglio/datasets/raw/master/human_activity/simplifiedhuarus/train.csv

In [2]:
import pandas as pd

data = pd.read_csv('./train.csv', sep=',')

data = data.drop(columns="rn")

pd.set_option('display.max_columns', 500)     # Make sure we can see all of the columns
pd.set_option('display.max_rows', 50)         # Keep the output on one page
data[:10] # Show the first 10 lines


,activity,tBodyAcc.mean.X,tBodyAcc.mean.Y,tBodyAcc.mean.Z,tBodyAcc.std.X,tBodyAcc.std.Y,tBodyAcc.std.Z,tBodyAcc.mad.X,tBodyAcc.mad.Y,tBodyAcc.mad.Z,tBodyAcc.max.X,tBodyAcc.max.Y,tBodyAcc.max.Z,tBodyAcc.min.X,tBodyAcc.min.Y,tBodyAcc.min.Z,tBodyAcc.sma,tBodyAcc.energy.X,tBodyAcc.energy.Y,tBodyAcc.energy.Z,tBodyAcc.iqr.X,tBodyAcc.iqr.Y,tBodyAcc.iqr.Z,tBodyAcc.entropy.X,tBodyAcc.entropy.Y,tBodyAcc.entropy.Z,tBodyAcc.arCoeff.X.1,tBodyAcc.arCoeff.X.2,tBodyAcc.arCoeff.X.3,tBodyAcc.arCoeff.X.4,tBodyAcc.arCoeff.Y.1,tBodyAcc.arCoeff.Y.2,tBodyAcc.arCoeff.Y.3,tBodyAcc.arCoeff.Y.4,tBodyAcc.arCoeff.Z.1,tBodyAcc.arCoeff.Z.2,tBodyAcc.arCoeff.Z.3,tBodyAcc.arCoeff.Z.4,tBodyAcc.correlation.X.Y,tBodyAcc.correlation.X.Z,tBodyAcc.correlation.Y.Z,tGravityAcc.mean.X,tGravityAcc.mean.Y,tGravityAcc.mean.Z,tGravityAcc.std.X,tGravityAcc.std.Y,tGravityAcc.std.Z,tGravityAcc.mad.X,tGravityAcc.mad.Y,tGravityAcc.mad.Z,tGravityAcc.max.X,tGravityAcc.max.Y,tGravityAcc.max.Z,tGravityAcc.min.X,tGravityAcc.min.Y,tGravityAcc.min.Z,tGravityAcc.sma,tGravityAcc.energy.X,tGravityAcc.energy.Y,tGravityAcc.energy.Z,tGravityAcc.iqr.X,tGravityAcc.iqr.Y,tGravityAcc.iqr.Z,tGravityAcc.entropy.X,tGravityAcc.entropy.Y,tGravityAcc.entropy.Z,tGravityAcc.arCoeff.X.1,tGravityAcc.arCoeff.X.2,tGravityAcc.arCoeff.X.3,tGravityAcc.arCoeff.X.4,tGravityAcc.arCoeff.Y.1,tGravityAcc.arCoeff.Y.2,tGravityAcc.arCoeff.Y.3,tGravityAcc.arCoeff.Y.4,tGravityAcc.arCoeff.Z.1,tGravityAcc.arCoeff.Z.2,tGravityAcc.arCoeff.Z.3,tGravityAcc.arCoeff.Z.4,tGravityAcc.correlation.X.Y,tGravityAcc.correlation.X.Z,tGravityAcc.correlation.Y.Z,tBodyAccJerk.mean.X,tBodyAccJerk.mean.Y,tBodyAccJerk.mean.Z,tBodyAccJerk.std.X,tBodyAccJerk.std.Y,tBodyAccJerk.std.Z,tBodyAccJerk.mad.X,tBodyAccJerk.mad.Y,tBodyAccJerk.mad.Z,tBodyAccJerk.max.X,tBodyAccJerk.max.Y,tBodyAccJerk.max.Z,tBodyAccJerk.min.X,tBodyAccJerk.min.Y,tBodyAccJerk.min.Z,tBodyAccJerk.sma,tBodyAccJerk.energy.X,tBodyAccJerk.energy.Y,tBodyAccJerk.energy.Z,tBodyAccJerk.iqr.X,tBodyAccJerk.iqr.Y,tBodyAccJerk.iqr.Z,tBodyAccJerk.entropy.X,tBodyAccJerk.entropy.Y,tBodyAccJerk.entropy.Z,tBodyAccJerk.arCoeff.X.1,tBodyAccJerk.arCoeff.X.2,tBodyAccJerk.arCoeff.X.3,tBodyAccJerk.arCoeff.X.4,tBodyAccJerk.arCoeff.Y.1,tBodyAccJerk.arCoeff.Y.2,tBodyAccJerk.arCoeff.Y.3,tBodyAccJerk.arCoeff.Y.4,tBodyAccJerk.arCoeff.Z.1,tBodyAccJerk.arCoeff.Z.2,tBodyAccJerk.arCoeff.Z.3,tBodyAccJerk.arCoeff.Z.4,tBodyAccJerk.correlation.X.Y,tBodyAccJerk.correlation.X.Z,tBodyAccJerk.correlation.Y.Z,tBodyGyro.mean.X,tBodyGyro.mean.Y,tBodyGyro.mean.Z,tBodyGyro.std.X,tBodyGyro.std.Y,tBodyGyro.std.Z,tBodyGyro.mad.X,tBodyGyro.mad.Y,tBodyGyro.mad.Z,tBodyGyro.max.X,tBodyGyro.max.Y,tBodyGyro.max.Z,tBodyGyro.min.X,tBodyGyro.min.Y,tBodyGyro.min.Z,tBodyGyro.sma,tBodyGyro.energy.X,tBodyGyro.energy.Y,tBodyGyro.energy.Z,tBodyGyro.iqr.X,tBodyGyro.iqr.Y,tBodyGyro.iqr.Z,tBodyGyro.entropy.X,tBodyGyro.entropy.Y,tBodyGyro.entropy.Z,tBodyGyro.arCoeff.X.1,tBodyGyro.arCoeff.X.2,tBodyGyro.arCoeff.X.3,tBodyGyro.arCoeff.X.4,tBodyGyro.arCoeff.Y.1,tBodyGyro.arCoeff.Y.2,tBodyGyro.arCoeff.Y.3,tBodyGyro.arCoeff.Y.4,tBodyGyro.arCoeff.Z.1,tBodyGyro.arCoeff.Z.2,tBodyGyro.arCoeff.Z.3,tBodyGyro.arCoeff.Z.4,tBodyGyro.correlation.X.Y,tBodyGyro.correlation.X.Z,tBodyGyro.correlation.Y.Z,tBodyGyroJerk.mean.X,tBodyGyroJerk.mean.Y,tBodyGyroJerk.mean.Z,tBodyGyroJerk.std.X,tBodyGyroJerk.std.Y,tBodyGyroJerk.std.Z,tBodyGyroJerk.mad.X,tBodyGyroJerk.mad.Y,tBodyGyroJerk.mad.Z,tBodyGyroJerk.max.X,tBodyGyroJerk.max.Y,tBodyGyroJerk.max.Z,tBodyGyroJerk.min.X,tBodyGyroJerk.min.Y,tBodyGyroJerk.min.Z,tBodyGyroJerk.sma,tBodyGyroJerk.energy.X,tBodyGyroJerk.energy.Y,tBodyGyroJerk.energy.Z,tBodyGyroJerk.iqr.X,tBodyGyroJerk.iqr.Y,tBodyGyroJerk.iqr.Z,tBodyGyroJerk.entropy.X,tBodyGyroJerk.entropy.Y,tBodyGyroJerk.entropy.Z,tBodyGyroJerk.arCoeff.X.1,tBodyGyroJerk.arCoeff.X.2,tBodyGyroJerk.arCoeff.X.3,tBodyGyroJerk.arCoeff.X.4,tBodyGyroJerk.arCoeff.Y.1,tBodyGyroJerk.arCoeff.Y.2,tBodyGyroJerk.arCoeff.Y.3,tBodyGyroJerk.arCoeff.Y.4,tBodyGyroJerk.arCoeff.Z.1,tBodyGyroJerk.arCo

# Split
Vamos a guardarnos un 5% del dataset para hacer inferencias sobre el modelo final.
El 95% lo vamos a subir a un bucket S3 para que Autopilot pueda trabajar sobre el.

In [3]:
import numpy as np

train_data, test_data, _ = np.split(data.sample(frac=1, random_state=123), 
                                                  [int(0.95 * len(data)), int(len(data))])  

# Save to CSV files
train_data.to_csv('had-autopilot-train.csv', index=False, header=True, sep=',') # Need to keep column names
test_data.to_csv('had-autopilot-test.csv', index=False, header=True, sep=',')

In [4]:
%%sh

ls -l *.csv

-rw-r--r-- 1 root root   984094 Mar  1 14:52 had-autopilot-test.csv
-rw-r--r-- 1 root root 18297797 Mar  1 14:52 had-autopilot-train.csv
-rw-r--r-- 1 root root 12859056 Mar  1 14:52 train.csv


# Upload a Amazon S3

In [5]:
import sagemaker

prefix = 'sagemaker/DEMO-autopilot-had-dataset/input'
sess   = sagemaker.Session()

uri = sess.upload_data(path="had-autopilot-train.csv", key_prefix=prefix)


# Iniciar el Autopilot en "Create Experiment"
Con los siguientes datos:

In [6]:
print("1. Experiment Name:")
print("[Por ejemplo] Autopilot-HAC-DEMO")
print("\n2. S3 location of input data: ")
print(uri)
print("\n3. Target attribute name:")
print("activity")
print("\n4. S3 location for output data: ")
print("s3://sagemaker-us-east-2-XXXXXXXXXXXX/sagemaker/DEMO-autopilot-had-dataset/output")

1. Experiment Name:
[Por ejemplo] Autopilot-HAC-DEMO

2. S3 location of input data: 
s3://sagemaker-us-east-2-XXXXXXXXXXXX/sagemaker/DEMO-autopilot-had-dataset/input/had-autopilot-train.csv

3. Target attribute name:
activity

4. S3 location for output data: 
s3://sagemaker-us-east-2-XXXXXXXXXXXX/sagemaker/DEMO-autopilot-had-dataset/output
